In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [2]:
# using pytorch native AMP
from torch.cuda.amp import GradScaler
from torch.cuda.amp import autocast

In [3]:
# using Tensorboard
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(log_dir="/data/runs")

In [4]:
!pip install wandb
# wandb login

In [5]:
import wandb
wandb.login()

wandb: Currently logged in as: jyotik (use `wandb login --relogin` to force relogin)


True

In [6]:
wandb.init(project="hw09-w251-ddp")

In [7]:
import torch.distributed as dist

In [8]:
# Assume that this notebook only sees one GPU.
GPU=0

In [9]:
SEED=1

In [10]:
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [11]:
torch.cuda.device_count()

1

In [12]:
START_EPOCH = 0

In [13]:
# ARCH = 'resnet18'
ARCH = 'resnet18'
EPOCHS = 200
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
PRINT_FREQ = 50
TRAIN_BATCH=256
VAL_BATCH=256
WORKERS=8
TRAINDIR="/data/train"
VALDIR="/data/val"

In [14]:
wandb.init(config={"epochs": EPOCHS, "batch_size": TRAIN_BATCH, "momentum": MOMENTUM, "WEIGHT_DECAY": WEIGHT_DECAY, "arch": ARCH})

In [15]:
TRAINDIR="/data/train"
VALDIR="/data/val"

In [16]:
if not torch.cuda.is_available():
    print('GPU not detected.. did you pass through your GPU?')

In [17]:
# how many processes in cluster?
WORLD_SIZE = 2
BACKEND = 'nccl'
# where is the master?
#URL = 'tcp://13.57.216.199:1234'
URL = 'tcp://13.57.177.94:1234'

In [18]:
#what is my rank?
#RANK = 0
RANK = 1

In [19]:
dist.init_process_group(backend = BACKEND, init_method= URL,
                                world_size= WORLD_SIZE, rank=RANK)

In [20]:
#torch.cuda.set_device('cpu')
torch.cuda.set_device(GPU)

In [21]:
cudnn.benchmark = True

In [22]:
global_step = 0

In [23]:
def train(train_loader, model, criterion, optimizer, epoch):
    global global_step
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # Grad Scaler
    scaler = GradScaler()
    
    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if GPU is not None:
            images = images.cuda(GPU, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(GPU, non_blocking=True)

        # compute output
        with autocast():
            output = model(images)
            loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        #optimizer.zero_grad()
        #loss.backward()
        #optimizer.step()
        
        # use the scaler
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        wandb.log({"Loss/train": loss, 'acc1/train': top1.avg, 'acc5/train': top5.avg})
        
        writer.add_scalar("Loss/train", loss, global_step = global_step)
        writer.add_scalar("acc1/train", top1.avg, global_step = global_step)
        writer.add_scalar("acc5/train", top5.avg, global_step = global_step)
        global_step = global_step + 1

        if i % PRINT_FREQ == 0:
            progress.display(i)

In [24]:
def validate(val_loader, model, criterion):
    global global_step
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            if GPU is not None:
                images = images.cuda(GPU, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(GPU, non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    writer.add_scalar("Loss/val", losses.avg, global_step = global_step)
    writer.add_scalar("acc1/val", top1.avg, global_step = global_step)
    writer.add_scalar("acc5/val", top5.avg, global_step = global_step)
    
    wandb.log({"Loss/val": losses.avg, 'acc1/val': top1.avg, 'acc5/val': top5.avg})
    return top1.avg

In [25]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [26]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [27]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [28]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [29]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [30]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]

In [31]:
normalize = transforms.Normalize(mean=imagenet_mean_RGB, std=imagenet_std_RGB)

In [32]:
IMG_SIZE = 224

In [33]:
NUM_CLASSES = 1000

In [34]:
model = models.__dict__[ARCH]()

In [35]:
inf = model.fc.in_features

In [36]:
model.fc = nn.Linear(inf, NUM_CLASSES)

In [37]:
model.cuda(GPU)
model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[GPU])
# model = torch.nn.parallel.DistributedDataParallel(model)

In [38]:
criterion = nn.CrossEntropyLoss().cuda(GPU)

In [39]:
optimizer = torch.optim.SGD(model.parameters(), LR,
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)

In [40]:
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [41]:
#transform_train = transforms.Compose([
#    transforms.Resize(256),
#    transforms.RandomResizedCrop(224),
#    transforms.CenterCrop(224),
#    transforms.RandomHorizontalFlip(),
#    transforms.ToTensor(),
#    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
#])

In [42]:
#train_dataset = datasets.ImageFolder(
#    TRAINDIR, transform=transform_train)

In [43]:
train_dataset = datasets.ImageFolder(
    TRAINDIR,
    transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ])
)

In [44]:
#transform_val = transforms.Compose([
#    transforms.Resize(256),
#    transforms.CenterCrop(224),
#    transforms.ToTensor(),
#    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
#])

In [45]:
#val_dataset = datasets.ImageFolder(
#    VALDIR, transform=transform_val)

In [46]:
val_dataset = datasets.ImageFolder(
    VALDIR,
    transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ])
)

In [47]:
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=TRAIN_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=torch.utils.data.distributed.DistributedSampler(train_dataset))

In [48]:
val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=VAL_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=torch.utils.data.distributed.DistributedSampler(val_dataset)) 

In [49]:
best_acc1 = 0

In [50]:
%%time
for epoch in range(START_EPOCH, 2):
    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()))
    writer.add_scalar("lr", scheduler.get_last_lr()[0], global_step = global_step)
    wandb.log({'lr': scheduler.get_last_lr()[0]})

/opt/conda/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1153.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: [0][   0/2503]	Time 14.148 (14.148)	Data  3.674 ( 3.674)	Loss 7.0252e+00 (7.0252e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.39 (  0.39)
Epoch: [0][  50/2503]	Time  0.371 ( 0.624)	Data  0.002 ( 0.074)	Loss 6.7622e+00 (6.9440e+00)	Acc@1   1.17 (  0.21)	Acc@5   3.91 (  1.02)
Epoch: [0][ 100/2503]	Time  0.345 ( 0.491)	Data  0.002 ( 0.039)	Loss 6.6621e+00 (6.8187e+00)	Acc@1   0.78 (  0.41)	Acc@5   2.34 (  1.71)
Epoch: [0][ 150/2503]	Time  0.347 ( 0.446)	Data  0.002 ( 0.027)	Loss 6.4812e+00 (6.7205e+00)	Acc@1   1.17 (  0.57)	Acc@5   3.91 (  2.29)
Epoch: [0][ 200/2503]	Time  0.367 ( 0.424)	Data  0.002 ( 0.021)	Loss 6.4402e+00 (6.6353e+00)	Acc@1   0.00 (  0.80)	Acc@5   3.52 (  2.95)
Epoch: [0][ 250/2503]	Time  0.344 ( 0.411)	Data  0.002 ( 0.017)	Loss 6.1696e+00 (6.5514e+00)	Acc@1   1.95 (  1.00)	Acc@5   6.64 (  3.63)


/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


Epoch: [0][ 300/2503]	Time  0.343 ( 0.402)	Data  0.002 ( 0.015)	Loss 6.1438e+00 (6.4739e+00)	Acc@1   2.73 (  1.19)	Acc@5   5.08 (  4.26)
Epoch: [0][ 350/2503]	Time  0.378 ( 0.395)	Data  0.003 ( 0.013)	Loss 6.1019e+00 (6.4061e+00)	Acc@1   2.34 (  1.40)	Acc@5   8.20 (  4.91)
Epoch: [0][ 400/2503]	Time  0.349 ( 0.391)	Data  0.002 ( 0.012)	Loss 5.7108e+00 (6.3431e+00)	Acc@1   3.12 (  1.60)	Acc@5  12.11 (  5.49)
Epoch: [0][ 450/2503]	Time  0.343 ( 0.387)	Data  0.002 ( 0.011)	Loss 5.7403e+00 (6.2829e+00)	Acc@1   2.73 (  1.79)	Acc@5   9.38 (  6.13)
Epoch: [0][ 500/2503]	Time  0.373 ( 0.385)	Data  0.003 ( 0.010)	Loss 5.7713e+00 (6.2242e+00)	Acc@1   1.95 (  2.00)	Acc@5  10.94 (  6.78)
Epoch: [0][ 550/2503]	Time  0.389 ( 0.382)	Data  0.004 ( 0.009)	Loss 5.6150e+00 (6.1699e+00)	Acc@1   3.52 (  2.21)	Acc@5  13.28 (  7.36)
Epoch: [0][ 600/2503]	Time  0.349 ( 0.380)	Data  0.002 ( 0.009)	Loss 5.5668e+00 (6.1193e+00)	Acc@1   4.30 (  2.43)	Acc@5  14.84 (  7.95)
Epoch: [0][ 650/2503]	Time  0.346 ( 0.379

/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


Epoch: [1][ 300/2503]	Time  0.369 ( 0.375)	Data  0.002 ( 0.013)	Loss 4.1487e+00 (3.9719e+00)	Acc@1  19.53 ( 20.94)	Acc@5  39.84 ( 43.12)
Epoch: [1][ 350/2503]	Time  0.348 ( 0.373)	Data  0.002 ( 0.012)	Loss 4.0946e+00 (3.9610e+00)	Acc@1  20.70 ( 21.11)	Acc@5  40.23 ( 43.39)
Epoch: [1][ 400/2503]	Time  0.347 ( 0.371)	Data  0.002 ( 0.011)	Loss 3.8892e+00 (3.9562e+00)	Acc@1  25.00 ( 21.22)	Acc@5  44.92 ( 43.51)
Epoch: [1][ 450/2503]	Time  0.347 ( 0.370)	Data  0.002 ( 0.010)	Loss 3.8615e+00 (3.9478e+00)	Acc@1  24.22 ( 21.39)	Acc@5  48.44 ( 43.69)
Epoch: [1][ 500/2503]	Time  0.344 ( 0.369)	Data  0.002 ( 0.009)	Loss 3.7865e+00 (3.9369e+00)	Acc@1  26.17 ( 21.55)	Acc@5  50.39 ( 43.93)
Epoch: [1][ 550/2503]	Time  0.354 ( 0.368)	Data  0.002 ( 0.008)	Loss 3.5855e+00 (3.9243e+00)	Acc@1  25.78 ( 21.74)	Acc@5  47.27 ( 44.19)
Epoch: [1][ 600/2503]	Time  0.375 ( 0.367)	Data  0.002 ( 0.008)	Loss 3.6673e+00 (3.9120e+00)	Acc@1  28.12 ( 21.95)	Acc@5  48.83 ( 44.42)
Epoch: [1][ 650/2503]	Time  0.344 ( 0.367